In [1]:
import pandas
import matplotlib
import collections
import seaborn
import re

%matplotlib inline

seaborn.set_style('whitegrid')
seaborn.set_palette('colorblind')

In [2]:
ecosystems = ['npm', 'cran']

COLORS = {ecos: seaborn.color_palette()[i] for i, ecos in enumerate(ecosystems)}
FIG_SIZE = (6, 3)
SHOW_TITLE = True

REGEX_FIXED_VERSION = re.compile(r'\^|~|>|<|\*|\.x')

In [3]:
def savefig(fig, name):
    # (Un)comment next line to (enable) disable export
    return
    return fig.savefig('../figures/{}.pdf'.format(name), bbox_inches='tight')

In [4]:
def snapshot(date, packages, dependencies):
    """
    Return a 2-uple (packages, dependencies) of dataframes that only contain
    information that were available and valid at given date.
    """
    date = pandas.to_datetime(date)
    pkg = packages[packages['time'] <= date].sort_values('time').groupby(level='package', sort=False).tail(1)
    deps = dependencies.merge(pkg, how='inner', left_on=['package', 'version'], right_index=True)
    deps = deps.drop('time', axis=1)
    return (pkg, deps)

In [5]:
df_pkg = collections.OrderedDict()
df_deps = collections.OrderedDict()

for ecosystem in ecosystems: 
    pkg = pandas.read_csv('../data/{ecosystem}/packages.csv.gz'.format(ecosystem=ecosystem), 
                          usecols=['package', 'version', 'time'],
                          index_col=['package', 'version'],
                          parse_dates=['time'])
    deps = pandas.read_csv('../data/{ecosystem}/dependencies.csv.gz'.format(ecosystem=ecosystem),
                           usecols=['package', 'version', 'dependency', 'constraint'])

    df_pkg[ecosystem] = pkg
    df_deps[ecosystem] = deps

In [6]:
for ecos, deps in df_deps.items():
    deps['fixed'] = ~ deps['constraint'].str.contains(REGEX_FIXED_VERSION)
    deps['constraint'] = deps['constraint'].where(~deps['fixed'], deps['constraint'].str.replace('=', ''), axis=0)

# Overview

## Number of packages, versions, dependencies (overall/in snapshot)

In [7]:
df = pandas.DataFrame(index=ecosystems)

for ecos in ecosystems:
    latest_date = df_pkg[ecos]['time'].max()
    snapshot_pkg, snapshot_deps = snapshot(latest_date, df_pkg[ecos], df_deps[ecos])
    
    df.loc[ecos, 'packages'] = len(df_pkg[ecos].groupby(level=0))
    df.loc[ecos, 'versions'] = len(df_pkg[ecos])
    df.loc[ecos, 'dependencies'] = len(df_deps[ecos])
    
    df.loc[ecos, 'snapshot'] = latest_date
    df.loc[ecos, 'deps in snapshot'] = len(snapshot_deps)
    
df['versions_mean'] = df['packages'] / df['versions']
df['dependencies_mean'] = df['dependencies'] / df['versions']

df

,packages,versions,dependencies,snapshot,deps in snapshot,versions_mean,dependencies_mean
npm,317159.0,1927750.0,7644295.0,2016-06-28 14:35:28.717,728447.0,0.164523,3.965397
cran,9568.0,57530.0,128113.0,2016-04-26 20:52:26.000,22191.0,0.166313,2.226890


## Number and proportion of fixed constraints (overall/in snasphot)

In [8]:
df = pandas.DataFrame(index=ecosystems)
df_snapshot = pandas.DataFrame(index=ecosystems)

for ecos in ecosystems:
    latest_date = df_pkg[ecos]['time'].max()
    snapshot_pkg, snapshot_deps = snapshot(latest_date, df_pkg[ecos], df_deps[ecos])
    
    df.loc[ecos, 'versions'] = len(df_pkg[ecos])
    df.loc[ecos, 'versions_with_deps'] = len(df_deps[ecos].groupby('version'))
    df.loc[ecos, 'versions_with_fixed'] = len(df_deps[ecos].query('fixed == True').groupby('version'))
    df.loc[ecos, 'deps'] = len(df_deps[ecos])
    df.loc[ecos, 'deps_fixed'] = len(df_deps[ecos].query('fixed == True'))
    
    df_snapshot.loc[ecos, 'versions'] = len(snapshot_pkg)
    df_snapshot.loc[ecos, 'versions_with_deps'] = len(snapshot_deps.groupby('version'))
    df_snapshot.loc[ecos, 'versions_with_fixed'] = len(snapshot_deps.query('fixed == True').groupby('version'))
    df_snapshot.loc[ecos, 'deps'] = len(snapshot_deps)
    df_snapshot.loc[ecos, 'deps_fixed'] = len(snapshot_deps.query('fixed == True'))

df['p_fixed_deps'] = df['deps_fixed'] / df['deps']
df['p_versions_with_fixed'] = df['versions_with_fixed'] / df['versions']
df['p_versions_with_deps_with_fixed'] = df['versions_with_fixed'] / df['versions_with_deps']

df_snapshot['p_fixed_deps'] = df_snapshot['deps_fixed'] / df_snapshot['deps']
df_snapshot['p_versions_with_fixed'] = df_snapshot['versions_with_fixed'] / df_snapshot['versions']
df_snapshot['p_versions_with_deps_with_fixed'] = df_snapshot['versions_with_fixed'] / df_snapshot['versions_with_deps']

df

,versions,versions_with_deps,versions_with_fixed,deps,deps_fixed,p_fixed_deps,p_versions_with_fixed,p_versions_with_deps_with_fixed
npm,1927750.0,33301.0,21154.0,7644295.0,1420047.0,0.185766,0.010973,0.635236
cran,57530.0,5722.0,14.0,128113.0,19.0,0.000148,0.000243,0.002447


In [9]:
df_snapshot

,versions,versions_with_deps,versions_with_fixed,deps,deps_fixed,p_fixed_deps,p_versions_with_fixed,p_versions_with_deps_with_fixed
npm,317160.0,4605.0,2543.0,728447.0,115006.0,0.157878,0.008018,0.552226
cran,9568.0,1303.0,5.0,22191.0,6.0,0.000270,0.000523,0.003837


In [11]:
dates = pandas.date_range('2010-01-01', '2016-09-01', freq='6M') - pandas.Timedelta('30D')

df_t_versions = pandas.DataFrame(index=dates, columns=ecosystems)
df_t_deps = pandas.DataFrame(index=dates, columns=ecosystems)
df_t_versions_deps = pandas.DataFrame(index=dates, columns=ecosystems)
df_t_conflict = pandas.DataFrame(index=dates, columns=ecosystems)

divide_by_zero = lambda a, b: a / b if b > 0 else 0

for ecos in ecosystems:
    for date in dates: 
        pkg, deps = snapshot(date, df_pkg[ecos], df_deps[ecos])
        
        packages, dependencies = len(pkg), len(deps)
        packages_with_dependencies = len(deps.groupby('package'))
        fixed_dependencies = len(deps.query('fixed == True'))
        fixed_packages = len(deps.query('fixed == True').groupby('package'))
        
        df_t_versions.loc[date, ecos] = divide_by_zero(fixed_packages, packages)
        df_t_deps.loc[date, ecos] = divide_by_zero(fixed_dependencies, dependencies)
        df_t_versions_deps.loc[date, ecos] = divide_by_zero(fixed_packages, packages_with_dependencies)
        
        # Packages are in conflict if they have the same dependency but with distinct fixed versions
        conflicts = len(deps.query('fixed == True')
                         .merge(fixed, how='inner', on=['dependency'], suffixes=['_left', '_right'])
                         .query('package_left != package_right and constraint_left != constraint_right')
                         .groupby('package_left'))
        df_t_conflict.loc[date, ecos] = conflicts

KeyError: MemoryError()

In [ ]:
fig, axes = matplotlib.pyplot.subplots(nrows=4, sharex=True)

df_t_versions.plot(ax=axes[0], title='Packages with a fixed dependency')
df_t_deps.plot(ax=axes[1], title='Fixed dependencies')
df_t_versions_deps.plot(ax=axes[2], title='Packages with dependencies, and a fixed dependency')
df_t_conflicts.plot(ax=axes[3], title='Package in conflict (non-transitive)')